# 🎨 Diffusion Models for Image Generation

Welcome to the **cutting-edge of generative AI**! In this notebook, we'll explore diffusion models - the technology behind DALL-E 2, Midjourney, and Stable Diffusion. Generate stunning images from text prompts!

## What you'll learn:
- Diffusion processes and reverse denoising
- DDPM and DDIM sampling techniques
- Text-to-image generation with CLIP
- Stable Diffusion and latent space generation

Let's create amazing art with AI! 🚀

In [ ]:
# Install required packages (run once)
# !pip install diffusers transformers accelerate torch torchvision xformers

# Import libraries
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
from PIL import Image
import requests
from io import BytesIO

# Diffusion libraries
from diffusers import (
    StableDiffusionPipeline, DDPMPipeline, DDIMPipeline,
    UNet2DModel, DDPMScheduler, DDIMScheduler
)
from transformers import CLIPTextModel, CLIPTokenizer

import warnings
warnings.filterwarnings('ignore')

plt.style.use('seaborn-v0_8')
np.random.seed(42)
torch.manual_seed(42)

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

In [ ]:
# Load Stable Diffusion pipeline
MODEL_ID = "runwayml/stable-diffusion-v1-5"

# Load the pipeline
print("🚀 Loading Stable Diffusion pipeline...")
print("This may take a few minutes on first run...")

try:
    pipe = StableDiffusionPipeline.from_pretrained(
        MODEL_ID,
        torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
        safety_checker=None,  # Disable for demo
        requires_safety_checker=False
    )
    pipe = pipe.to(device)
    
    # Enable memory efficient attention
    if torch.cuda.is_available():
        pipe.enable_attention_slicing()
        pipe.enable_xformers_memory_efficient_attention()
    
    print("✅ Stable Diffusion loaded successfully!")
    
except Exception as e:
    print(f"⚠️ Could not load Stable Diffusion: {e}")
    print("Creating a simplified demo instead...")
    pipe = None

In [ ]:
# Text-to-Image Generation
def generate_images(prompts, num_images=1, guidance_scale=7.5, num_steps=20):
    """Generate images from text prompts"""
    
    if pipe is None:
        print("⚠️ Stable Diffusion not available. Creating placeholder images...")
        # Create placeholder images
        images = []
        for prompt in prompts:
            # Create a colorful placeholder
            img = np.random.rand(512, 512, 3) * 255
            img = img.astype(np.uint8)
            images.append(Image.fromarray(img))
        return images
    
    images = []
    
    for prompt in prompts:
        print(f"🎨 Generating: '{prompt}'")
        
        # Generate image
        with torch.autocast(device.type):
            result = pipe(
                prompt,
                num_images_per_prompt=num_images,
                guidance_scale=guidance_scale,
                num_inference_steps=num_steps,
                height=512,
                width=512
            )
        
        images.extend(result.images)
    
    return images

# Test prompts
test_prompts = [
    "A beautiful sunset over a mountain lake, digital art",
    "A cute robot reading a book in a cozy library",
    "A magical forest with glowing mushrooms and fireflies"
]

# Generate images
generated_images = generate_images(test_prompts, num_images=1)

# Display results
fig, axes = plt.subplots(1, len(test_prompts), figsize=(15, 5))
if len(test_prompts) == 1:
    axes = [axes]

for i, (prompt, img) in enumerate(zip(test_prompts, generated_images)):
    axes[i].imshow(img)
    axes[i].set_title(f"'{prompt[:30]}...'")
    axes[i].axis('off')

plt.suptitle('🎨 Text-to-Image Generation Results', fontsize=16)
plt.tight_layout()
plt.show()

print("✅ Image generation completed!")

In [ ]:
# Explore different guidance scales
def explore_guidance_scales(prompt, scales=[1.0, 5.0, 7.5, 10.0, 15.0]):
    """Explore how guidance scale affects generation"""
    
    if pipe is None:
        print("⚠️ Stable Diffusion not available for guidance scale exploration")
        return
    
    print(f"🎯 Exploring guidance scales for: '{prompt}'")
    
    images = []
    for scale in scales:
        print(f"Generating with guidance scale: {scale}")
        
        with torch.autocast(device.type):
            result = pipe(
                prompt,
                guidance_scale=scale,
                num_inference_steps=20,
                height=512,
                width=512
            )
        
        images.append(result.images[0])
    
    # Display results
    fig, axes = plt.subplots(1, len(scales), figsize=(20, 4))
    
    for i, (scale, img) in enumerate(zip(scales, images)):
        axes[i].imshow(img)
        axes[i].set_title(f'Guidance Scale: {scale}')
        axes[i].axis('off')
    
    plt.suptitle(f'🎯 Guidance Scale Effects: "{prompt}"', fontsize=16)
    plt.tight_layout()
    plt.show()

# Test guidance scales
explore_guidance_scales("A majestic dragon flying over a medieval castle")

In [ ]:
# Creative prompt exploration
creative_prompts = {
    "Artistic Styles": [
        "A cat in the style of Van Gogh's Starry Night",
        "A cityscape in cyberpunk style with neon lights",
        "A portrait in the style of Leonardo da Vinci"
    ],
    "Fantasy Scenes": [
        "A wizard's tower floating in the clouds",
        "An underwater city with mermaids and coral",
        "A steampunk airship flying through storm clouds"
    ],
    "Futuristic Concepts": [
        "A robot gardener tending to alien plants",
        "A space station orbiting a purple planet",
        "A holographic concert in a futuristic city"
    ]
}

print("🎨 Creative Prompt Exploration:")
print("=" * 50)

for category, prompts in creative_prompts.items():
    print(f"\n📂 {category}:")
    
    # Generate one image per category for demo
    sample_prompt = prompts[0]
    images = generate_images([sample_prompt], num_images=1)
    
    # Display
    plt.figure(figsize=(8, 8))
    plt.imshow(images[0])
    plt.title(f'{category}\n"{sample_prompt}"')
    plt.axis('off')
    plt.show()
    
    print(f"Generated: {sample_prompt}")
    print("-" * 30)

In [ ]:
# Demonstrate diffusion process visualization
def visualize_diffusion_process():
    """Visualize the diffusion denoising process"""
    
    if pipe is None:
        print("⚠️ Creating simulated diffusion process...")
        
        # Create a simple simulation of the diffusion process
        steps = [0, 5, 10, 15, 20]
        images = []
        
        # Start with noise
        base_image = np.random.randn(64, 64, 3) * 0.5 + 0.5
        
        for step in steps:
            # Simulate progressive denoising
            noise_level = (20 - step) / 20
            
            # Create a simple pattern that emerges
            x, y = np.meshgrid(np.linspace(-1, 1, 64), np.linspace(-1, 1, 64))
            pattern = np.sin(x * 3) * np.cos(y * 3)
            pattern = (pattern + 1) / 2  # Normalize to [0, 1]
            
            # Mix noise and pattern based on step
            noise = np.random.randn(64, 64) * noise_level
            signal = pattern * (1 - noise_level)
            
            combined = signal + noise * 0.3
            combined = np.clip(combined, 0, 1)
            
            # Convert to RGB
            rgb_image = np.stack([combined] * 3, axis=-1)
            images.append(rgb_image)
        
        # Display the process
        fig, axes = plt.subplots(1, len(steps), figsize=(15, 3))
        
        for i, (step, img) in enumerate(zip(steps, images)):
            axes[i].imshow(img)
            axes[i].set_title(f'Step {step}/20')
            axes[i].axis('off')
        
        plt.suptitle('🌊 Simulated Diffusion Denoising Process', fontsize=16)
        plt.tight_layout()
        plt.show()
        
        return
    
    # Real diffusion process visualization would require custom pipeline
    print("🌊 Diffusion Process Visualization")
    print("Note: Full process visualization requires custom implementation")
    print("The diffusion model progressively removes noise over multiple steps")
    
    # Show noise schedule
    scheduler = DDIMScheduler.from_pretrained(MODEL_ID, subfolder="scheduler")
    timesteps = scheduler.timesteps[:10]  # First 10 steps
    
    plt.figure(figsize=(10, 6))
    plt.plot(timesteps.cpu().numpy())
    plt.title('📊 Diffusion Timestep Schedule')
    plt.xlabel('Step')
    plt.ylabel('Timestep')
    plt.grid(True, alpha=0.3)
    plt.show()

# Visualize diffusion process
visualize_diffusion_process()

In [ ]:
# Interactive prompt generator
def interactive_prompt_generator():
    """Interactive text-to-image generation"""
    
    print("🎮 Interactive Image Generation")
    print("Enter prompts and generate amazing images!")
    print("Type 'quit' to exit\n")
    
    while True:
        try:
            prompt = input("Enter your prompt: ")
            
            if prompt.lower() == 'quit':
                break
            
            if not prompt.strip():
                continue
            
            # Get generation parameters
            print("\nGeneration options:")
            print("1. Fast (10 steps)")
            print("2. Balanced (20 steps)")
            print("3. High Quality (50 steps)")
            
            choice = input("Choose option (1-3, default=2): ").strip() or '2'
            
            # Set parameters based on choice
            if choice == '1':
                steps, guidance = 10, 7.5
            elif choice == '3':
                steps, guidance = 50, 10.0
            else:
                steps, guidance = 20, 7.5
            
            # Generate image
            print(f"\n🎨 Generating image for: '{prompt}'")
            print(f"Steps: {steps}, Guidance: {guidance}")
            
            images = generate_images([prompt], num_images=1, 
                                   guidance_scale=guidance, num_steps=steps)
            
            # Display result
            plt.figure(figsize=(8, 8))
            plt.imshow(images[0])
            plt.title(f'Generated: "{prompt}"')
            plt.axis('off')
            plt.show()
            
            # Save option
            save = input("Save image? (y/n): ").lower().startswith('y')
            if save:
                filename = f"generated_{len(prompt.split())}_words.png"
                images[0].save(filename)
                print(f"💾 Saved as {filename}")
            
            print("\n" + "="*50 + "\n")
            
        except KeyboardInterrupt:
            break
        except Exception as e:
            print(f"Error: {e}")
    
    print("Thanks for using the interactive generator!")

# Note: Uncomment the line below to run interactive mode
# interactive_prompt_generator()

In [ ]:
# Advanced techniques demonstration
def demonstrate_advanced_techniques():
    """Demonstrate advanced diffusion techniques"""
    
    print("🚀 Advanced Diffusion Techniques")
    print("=" * 40)
    
    # 1. Negative prompting
    print("\n1. 🚫 Negative Prompting:")
    positive_prompt = "A beautiful landscape with mountains"
    negative_prompt = "blurry, low quality, distorted"
    
    if pipe is not None:
        with torch.autocast(device.type):
            result = pipe(
                positive_prompt,
                negative_prompt=negative_prompt,
                guidance_scale=7.5,
                num_inference_steps=20
            )
        
        plt.figure(figsize=(8, 6))
        plt.imshow(result.images[0])
        plt.title(f'Positive: "{positive_prompt}"\nNegative: "{negative_prompt}"')
        plt.axis('off')
        plt.show()
    
    # 2. Seed control for reproducibility
    print("\n2. 🌱 Seed Control:")
    prompt = "A cute cat wearing a wizard hat"
    seeds = [42, 123, 456]
    
    if pipe is not None:
        fig, axes = plt.subplots(1, len(seeds), figsize=(15, 5))
        
        for i, seed in enumerate(seeds):
            generator = torch.Generator(device=device).manual_seed(seed)
            
            with torch.autocast(device.type):
                result = pipe(
                    prompt,
                    generator=generator,
                    guidance_scale=7.5,
                    num_inference_steps=20
                )
            
            axes[i].imshow(result.images[0])
            axes[i].set_title(f'Seed: {seed}')
            axes[i].axis('off')
        
        plt.suptitle(f'🌱 Same Prompt, Different Seeds: "{prompt}"')
        plt.tight_layout()
        plt.show()
    
    # 3. Aspect ratio variations
    print("\n3. 📐 Aspect Ratio Variations:")
    prompt = "A serene Japanese garden with cherry blossoms"
    dimensions = [(512, 512), (768, 512), (512, 768)]
    labels = ["Square", "Landscape", "Portrait"]
    
    if pipe is not None:
        fig, axes = plt.subplots(1, len(dimensions), figsize=(15, 8))
        
        for i, ((width, height), label) in enumerate(zip(dimensions, labels)):
            with torch.autocast(device.type):
                result = pipe(
                    prompt,
                    width=width,
                    height=height,
                    guidance_scale=7.5,
                    num_inference_steps=20
                )
            
            axes[i].imshow(result.images[0])
            axes[i].set_title(f'{label}\n{width}x{height}')
            axes[i].axis('off')
        
        plt.suptitle(f'📐 Aspect Ratio Variations: "{prompt}"')
        plt.tight_layout()
        plt.show()
    
    print("\n✅ Advanced techniques demonstration completed!")

# Demonstrate advanced techniques
demonstrate_advanced_techniques()

# Final summary
print(f"\n📊 Diffusion Models Summary:")
print(f"Model: Stable Diffusion v1.5")
print(f"Device: {device}")
print(f"Available: {'Yes' if pipe is not None else 'No (Demo mode)'}")
print(f"Capabilities: Text-to-image, negative prompting, seed control")
print(f"Resolution: Up to 768x768 (higher with memory optimization)")

## 🎉 Congratulations!

You've successfully explored diffusion models for image generation! Here's what you've accomplished:

✅ **Diffusion Models**: Understood the denoising process  
✅ **Stable Diffusion**: Generated high-quality images from text  
✅ **Text-to-Image**: Created diverse artistic content  
✅ **Advanced Techniques**: Explored guidance, seeds, and negative prompts  
✅ **Creative Applications**: Generated art, landscapes, and fantasy scenes  

### 🚀 Next Steps:
1. Explore ControlNet for precise image control
2. Try DreamBooth for personalized generation
3. Experiment with inpainting and outpainting
4. Build custom diffusion models for specific domains

## 🎊 **COLLECTION COMPLETE!**

You've now completed all **12 Deep Learning Projects**! From basic neural networks to cutting-edge diffusion models, you've mastered:

- **Fundamentals**: Neural networks, CNNs, RNNs
- **Computer Vision**: Image classification, GANs, style transfer
- **NLP**: Text classification, Transformers, BERT, GPT-2
- **Generative AI**: VAEs, diffusion models
- **Time Series**: LSTM forecasting

**Congratulations on your deep learning journey! 🌟**